In [2]:
import json
import os
from tqdm import tqdm

In [21]:
paths =[]
for i,j,k in os.walk('./'):
    paths.extend([i+'/'+p for p in k if p.endswith('.jsonl')])

In [4]:
trans_lang_codes={
    'en':'eng',
    'hi':'hin',
    'pa':'pan',
    'gu':'guj',
    'or':'ori',
    'as':'ben',
    'te':'tel',
    'ml':'mal',
    'mr':'hin',
    'ta':'tam',
    'bn':'ben',
    'kn':'kan'
}

In [5]:
import os
os.environ['PYTHONPATH']='/scratch/datasets_v2.7/fairseq/:/scratch/datasets_v2.7/indicTrans/'
from indictrans import Transliterator


%cd indicTrans
from inference.engine import Model
en2indic_model = Model(expdir='/scratch/siri.venkata/en-indic')

/scratch/datasets_v2.7/indicTrans


Error No module named 'triton'
Error No module named 'triton'


Initializing vocab and bpe
Initializing model for translation


In [6]:
def get_transliteration_dict(all_tails,target="hin"):
    tl ={}
    trn = Transliterator(source='eng', target=target)
    for i in tqdm(all_tails):
        tl[i] = trn.transform(i)
    return tl
#     tt[i] = en2indic_model.translate_paragraph(i,'en','hi')

In [17]:
def get_translation_dict(all_tails,src='en',target='hi'):
    tt={}
    all_tails_list = list(all_tails)
#     print(len(all_tails_list))
    for i in tqdm(range(0,len(all_tails),500)):
        start,end = i,i+500
        original = all_tails_list[start:end]
        translated = en2indic_model.batch_translate(original,'en',target)
#         print(start,end)
#         print(original,translated)
        for k,v in zip(original,translated):
            tt[k] = v
    return tt

In [8]:
%cd ../

/scratch/datasets_v2.7


In [22]:
finaldata={}
for path in paths:
    with open(path,'r') as f:
        lines = [json.loads(line) for line in f.readlines()]
    data=[]
    for line in lines:
        if not sum([1 if fact[3] else 0 for fact in line['facts']]):
            data.append(line)
    print('Number of lines in ',path )
    print(len(data),len(lines))
    all_tails=set()
    for v_ in data:
        all_tails = all_tails.union(set([j[1] for j in v_['facts']]))
        
    curr_lang = path.split('/')[1]
    if curr_lang =='en':
        tl = {i:i for i in all_tails}
        tt = tl
    else:    
        tl = get_transliteration_dict(all_tails,target=trans_lang_codes[curr_lang])
        tt = get_translation_dict(all_tails,target=curr_lang)
    
#     try:
#         tl = get_transliteration_dict(all_tails,target=trans_lang_codes[curr_lang])
#     except:
#         tl = {i:i for i in all_tails}
    
#     try:
#         tt = get_translation_dict(all_tails,target=curr_lang)
#     except:
#         tt = {i:i for i in all_tails}
    curr_split = path.split('/')[-1].split('.')[0]
    
    json.dump(tt,open('./transformed/'+curr_lang+'_'+curr_split+'_translation.json','w'))
    json.dump(tl,open('./transformed/'+curr_lang+'_'+curr_split+'_transliteration.json','w'))
    finaldata[path]=data
    !scp -r transformed siri.venkata@ada:/share1/siri.venkata/datasets_v2.7/
json.dump(finaldata,open('./transformed/finaldata.json','w'))
!scp -r transformed siri.venkata@ada:/share1/siri.venkata/datasets_v2.7/

Number of lines in  ./hi/val.jsonl
4417 5659


100%|███████████████████████████████████████████████████████████| 7/7 [00:14<00:00,  2.00s/it]


gu_test_transliteration.json                  100%   53KB  30.1MB/s   00:00    
or_test_translation.json                      100%   19KB  20.6MB/s   00:00    
kn_val_transliteration.json                   100%  135KB  51.3MB/s   00:00    
en_val_translation.json                       100%  437KB  69.2MB/s   00:00    
or_val_translation.json                       100%   46KB  43.9MB/s   00:00    
as_val_translation.json                       100%   51KB  45.3MB/s   00:00    
mr_val_translation.json                       100%   44KB  41.9MB/s   00:00    
ml_test_transliteration.json                  100%   71KB  54.5MB/s   00:00    
ml_train_transliteration.json                 100% 1966KB  95.1MB/s   00:00    
hi_test_transliteration.json                  100%   59KB  52.6MB/s   00:00    
ml_test_translation.json                      100%   41KB  40.4MB/s   00:00    
te_train_transliteration.json                 100%  662KB 103.3MB/s   00:00    
as_train_translation.json               

100%|█████████████████████████████████████████████████████████| 35/35 [01:13<00:00,  2.09s/it]


gu_test_transliteration.json                  100%   53KB  38.3MB/s   00:00    
or_test_translation.json                      100%   19KB  19.6MB/s   00:00    
kn_val_transliteration.json                   100%  135KB  50.1MB/s   00:00    
en_val_translation.json                       100%  437KB  63.3MB/s   00:00    
or_val_translation.json                       100%   46KB  35.3MB/s   00:00    
as_val_translation.json                       100%   51KB  37.2MB/s   00:00    
mr_val_translation.json                       100%   44KB  39.5MB/s   00:00    
ml_test_transliteration.json                  100%   71KB  47.6MB/s   00:00    
ml_train_transliteration.json                 100% 1966KB  79.3MB/s   00:00    
hi_test_transliteration.json                  100%   59KB  41.0MB/s   00:00    
ml_test_translation.json                      100%   41KB  37.7MB/s   00:00    
te_train_transliteration.json                 100%  662KB  80.2MB/s   00:00    
as_train_translation.json               

...

100%|█████████████████████████████████████████████████████████| 31/31 [01:13<00:00,  2.38s/it]


gu_test_transliteration.json                  100%   53KB  31.4MB/s   00:00    
or_test_translation.json                      100%   19KB  21.7MB/s   00:00    
kn_val_transliteration.json                   100%  135KB  49.7MB/s   00:00    
en_val_translation.json                       100%  437KB  78.8MB/s   00:00    
or_val_translation.json                       100%   74KB  63.0MB/s   00:00    
as_val_translation.json                       100%   60KB  45.9MB/s   00:00    
mr_val_translation.json                       100%  160KB  71.6MB/s   00:00    
ml_test_transliteration.json                  100%   71KB  61.3MB/s   00:00    
ml_train_transliteration.json                 100% 1966KB  97.9MB/s   00:00    
hi_test_transliteration.json                  100%   59KB  49.5MB/s   00:00    
ml_test_translation.json                      100%   68KB  57.9MB/s   00:00    
te_train_transliteration.json                 100%  662KB 100.6MB/s   00:00    
as_train_translation.json               

100%|███████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.39s/it]


gu_test_transliteration.json                  100%   53KB  40.1MB/s   00:00    
or_test_translation.json                      100%   19KB  22.5MB/s   00:00    
kn_val_transliteration.json                   100%  135KB  65.4MB/s   00:00    
en_val_translation.json                       100%  437KB  80.3MB/s   00:00    
or_val_translation.json                       100%   74KB  66.9MB/s   00:00    
as_val_translation.json                       100%   60KB  56.2MB/s   00:00    
mr_val_translation.json                       100%  160KB  81.1MB/s   00:00    
ml_test_transliteration.json                  100%   71KB  59.2MB/s   00:00    
ml_train_transliteration.json                 100% 1966KB 107.9MB/s   00:00    
hi_test_transliteration.json                  100%   59KB  56.1MB/s   00:00    
ml_test_translation.json                      100%   68KB  60.0MB/s   00:00    
te_train_transliteration.json                 100%  662KB 102.3MB/s   00:00    
as_train_translation.json               

ta_test_transliteration.json                  100%   58KB  52.0MB/s   00:00    
as_test_translation.json                      100%   71KB  58.7MB/s   00:00    
ml_val_transliteration.json                   100%  363KB  95.9MB/s   00:00    
ml_val_translation.json                       100%  349KB  94.9MB/s   00:00    
hi_val_translation.json                       100%  314KB  94.9MB/s   00:00    
mr_train_transliteration.json                 100%  892KB 105.6MB/s   00:00    
as_val_transliteration.json                   100%   59KB  57.3MB/s   00:00    
kn_train_transliteration.json                 100%  724KB 103.5MB/s   00:00    
mr_test_translation.json                      100%   76KB  64.5MB/s   00:00    
te_val_transliteration.json                   100%  129KB  74.7MB/s   00:00    
bn_test_transliteration.json                  100%   39KB  45.6MB/s   00:00    
pa_val_translation.json                       100%  188KB  83.7MB/s   00:00    
te_test_translation.json                

In [23]:
json.dump(finaldata,open('./transformed/finaldata.json','w'))


In [14]:
[ for i in finaldata[paths[0]] for fact in i['facts']]

[{'qid': 'Q448360',
  'entity_name': 'P. V. Sindhu',
  'sentence': 'సైనా నెహ్వాల్ 2012 ఒలంపిక్స్ లో కాంస్యపతకం సాధించిన తరువాత బ్యాడ్మింటన్ లో పతకం సాధించిన రెండో క్రీడాకారిణి సింధు.',
  'native_sentence_section': 'introduction',
  'translated_sentence': 'Sindhu is the second Indian after Saina Nehwal to win an Olympic medal in badminton after 2012.',
  'sent_index': 7,
  'facts': [['sport', 'badminton', [], False],
   ['occupation', 'badminton player', [], False]],
  'sent_score': '0.62',
  'translated_facts': [['క్రీడలు', 'బాడ్మింటన్', [], False],
   ['వృత్తి', 'బ్యాడ్మింటన్ ప్లేయర్', [], False]],
  'coverage_score': '0.4845',
  'lang': 'te',
  'translated_sentence_section': 'introduction',
  'complete_coverage': 0,
  'order': [0, 1],
  'role_spec_order': [0, 1]},
 {'qid': 'Q15146789',
  'entity_name': 'Anuradha Lohia',
  'sentence': 'కలకత్తా విశ్వవిద్యాలయంలోని ప్రెసిడెన్సీ.కళాశాల నుండి ఫిజియాలజీ, ఎం. యస్సీ.చేసింది. కలకత్తా విశ్వవిద్యాలయానికి చెద్మిన రాజాబజార్ సైన్స్ కాలేజీ నుండి ఫిజ

In [11]:
!du -sh transformed/*

64K	transformed/as_test_translation.json
72K	transformed/as_test_transliteration.json
52K	transformed/as_train_translation.json
312K	transformed/as_train_transliteration.json
52K	transformed/as_val_translation.json
60K	transformed/as_val_transliteration.json
44K	transformed/bn_test_translation.json
40K	transformed/bn_test_transliteration.json
56K	transformed/bn_train_translation.json
3.1M	transformed/bn_train_transliteration.json
60K	transformed/bn_val_translation.json
660K	transformed/bn_val_transliteration.json
32K	transformed/en_test_translation.json
32K	transformed/en_test_transliteration.json
2.3M	transformed/en_train_translation.json
2.3M	transformed/en_train_transliteration.json
440K	transformed/en_val_translation.json
440K	transformed/en_val_transliteration.json
551M	transformed/finaldata.json
44K	transformed/gu_test_translation.json
56K	transformed/gu_test_transliteration.json
48K	transformed/gu_train_translation.json
280K	transformed/gu_train_transliteration.json
48K	transfor

In [40]:
all_relas=[]
count=0
count2=0
for i in tqdm(finaldata):
    for data in finaldata[i]:
        count2+=1
        for fact in data['facts']:
                all_relas.append(fact[0])
                count+=1
        all_relas = list(set(all_relas))
    print(len(all_relas))
            

  0%|                                                                  | 0/36 [00:00<?, ?it/s]

102


 14%|████████                                                  | 5/36 [00:00<00:02, 12.91it/s]

182
183
184
200
200
200
203
203
206


 36%|████████████████████▌                                    | 13/36 [00:00<00:00, 32.16it/s]

...

254
254
254
254
254
256


 89%|██████████████████████████████████████████████████▋      | 32/36 [00:02<00:00,  9.81it/s]

262
262
262


100%|█████████████████████████████████████████████████████████| 36/36 [00:03<00:00, 11.75it/s]

265
265


In [39]:
count

854556

In [41]:
count2

458834

# OpenIE

In [ ]:
from openie import StanfordOpenIE

# https://stanfordnlp.github.io/CoreNLP/openie.html#api
# Default value of openie.affinity_probability_cap was 1/3.
properties = {
    'openie.affinity_probability_cap': 2 / 3,
}

In [ ]:
import json
from tqdm import tqdm
finaldata = json.load(open('transformed/finaldata.json','r'))

In [ ]:
newdata={}
with StanfordOpenIE(properties=properties) as client:
    for i in finaldata:
        if i.endswith("test.jsonl"):
            curr_dicts=[]
            for dat in tqdm(finaldata[i]):
                new_dict = dat
                triples = client.annotate(dat['translated_sentence'])
                new_dict['extracted_facts'] = [[ext['relation'],ext['object']] for ext in triples]
                curr_dicts.append(new_dict)
            newdata[i] = curr_dicts
        else:
            newdata[i] = finaldata[i]

In [ ]:
json.dump(newdata,open('transformed/finaldata.json','w'))

# Nounchunks

In [ ]:
import spacy

NER = spacy.load("en_core_web_sm")

In [ ]:
from icecream import ic
import re
import datefinder
datefinder.find_dates(text)

date_finder = datefinder.DateFinder(base_date=None, first="month")

In [ ]:



# samp=301
# ic(newdata['./te/test.jsonl'][samp])
# text = newdata['./te/test.jsonl'][samp]['translated_sentence']

    
    
    
def get_ents(text):
    text,chunks = remove_dates(text)
#     print(text,chunks)
#     print('*'*100)
    for chunk in NER(text).noun_chunks:
#         noun = chunk.root
#         print(chunk)
#         print(chunk.root,chunk.root.pos_)
#         print('**')
#         for tok in chunk:
#             print(tok,tok.pos_)
#         print('***************')

        if chunk.root.pos_=='NOUN':
            string = ''
            string2=''
            for tok in chunk:
                if tok.pos_=='PROPN':
                    string+=tok.text+' '
                elif tok.pos_=='ADJ':
                    string2+=tok.text+' '
            chunks.append(string)
            chunks.append(string2)
            chunks.append(chunk.root.text)
        elif chunk.root.pos_=='PRON':
            continue
        else:
            chunks.append(chunk.text)
    return [i for i in chunks if i!='']
                

    
# get_ents(text)

In [ ]:
# Cleaning any previous assignments
for i in newdata:
    for dat in tqdm(newdata[i]):
        dat['facts'] = [fact[:4] for fact in dat['facts']]

In [ ]:
def iou(a,b):
    return len(a.intersection(b))/len(a.union(b)) if len(a) and len(b) else 0
iou({1,2,3},{0,2,4})

In [ ]:
def cosinesimilarity(a,b):
    try:
        return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    except:
        return 0

In [ ]:
def get_sim_scores(x,A):
    return [iou(x[1],option[1])+cosinesimilarity(x[0],option[0])
                         for option in A]

## Text Encoding

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile
glove_file = datapath('glove.6B.300d.txt')# If glove vectors in default path
tmp_file = get_tmpfile("test_word2vec.txt")

In [ ]:
_ = glove2word2vec('glove.6B.300d.txt', tmp_file)# use glove file instead if glove vectors in path

In [ ]:
from gensim import models
model = models.KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:
import numpy as np
def encode_text(text):
    unmatched_words=[]
    vector=np.zeros(300)
    for i in text.split():
        try:
            vector+=model[i.lower()]
        except:
            unmatched_words.append(i.lower())
    return vector,unmatched_words

## Alignment

In [ ]:

def allignment(data):
    options=get_ents(data['translated_sentence'])
    tails = [i[1] for i in data['facts']]
    
    options_encoded =[encode_text(i) for i in options]
    head_encoded = encode_text(data['entity_name'])
    try:
        head_scores = get_sim_scores(head_encoded,options_encoded)
        rem_indices = [i for i,j in enumerate(head_scores) if j>0.7]
        del options_encoded[rem_indices]
        del options[rem_indices]
    except:
        pass
    tots=len(options)
    tails_encoded =[encode_text(i) for i in tails]
#     print('Length of tails ',len(tails_encoded))
    aligned=[]
    for index,i in enumerate(tails_encoded):
        try:
            tail_scores = get_sim_scores(i,options_encoded)
#             print(tails[index])
#             print(options)
#             print(tail_scores)
            match_index = [i for i,j in enumerate(tail_scores) if j>0.7][0]
            aligned.append(options[match_index])
            del options_encoded[match_index]
            del options[match_index]   
        except Exception as e:
#             print(e)
            aligned.append(-1)
#     print(aligned)
    new_dict = data
    new_dict['facts'] = [fact+[align] for fact,align in zip(data['facts'],aligned)]
    return (tots-len(options))/tots if tots!=0 else 1

data = newdata['./te/test.jsonl'][samp]
ic(allignment(data))
1

In [ ]:
precisions=[]
for i in newdata:
    print(i)
    if i.endswith('test.jsonl'):
        precisions.append([allignment(data) for data in tqdm(newdata[i])])
    else:
        pass

## Alignment Precision

In [ ]:
prcnt,ln=[],[]
index=0
for i in newdata:
    if i.endswith('test.jsonl'):
        for dat in newdata[i]:
            found = precisions[index]
            prcnt.append(np.mean(found))
            ln.append(len(found))
        index+=1

In [ ]:
sum([i*j for i,j in zip(prcnt,ln)])/sum(ln)

## Allignment Recall

In [ ]:
prcnt,ln=[],[]
for i in newdata:
    if i.endswith('test.jsonl'):
        for dat in newdata[i]:
            found = [1 if fact[-1]!=-1 else 0 for fact in dat['facts']]
            prcnt.append(np.mean(found))
            ln.append(len(found))

In [ ]:
sum([i*j for i,j in zip(prcnt,ln)])/sum(ln)

In [ ]:
json.dump(newdata,open('transformed/finaldata.json','w'))